In [ ]:
from google.cloud.bigquery import Client, QueryJobConfig
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

client = Client()

In [ ]:
sql_query = "SELECT * FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` LIMIT 10"

bigquery_response = client.query(sql_query)

df = bigquery_response.to_dataframe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sql_query = " \
    SELECT borough, COUNT(*) AS number_of_collisions \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    GROUP BY borough \
    ORDER BY number_of_collisions DESC \
"

bigquery_response = client.query(sql_query)

collisions_by_borough = bigquery_response.to_dataframe()

In [ ]:
collisions_by_borough.drop(index=0, inplace=True)

In [ ]:
collisions_by_borough

In [ ]:
plt.figure(figsize=(5, 3))
plt.bar(x=collisions_by_borough["borough"], 
        height=collisions_by_borough["number_of_collisions"],
        width=0.4, color="red", alpha=0.7, edgecolor="black")
plt.title("Collisions by Borough")
plt.xlabel("Borough")
plt.ylabel("Number of Collisions")
plt.grid(True)
plt.show()

### Which zipcodes are the most dangerous?

In [ ]:
sql_query = " \
    SELECT zip_code, COUNT(*) AS number_of_collisions \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    GROUP BY zip_code \
    ORDER BY number_of_collisions DESC \
"

bigquery_response = client.query(sql_query)

collisions_by_zipcode = bigquery_response.to_dataframe()

In [ ]:
collisions_by_zipcode.drop(index=0, inplace=True)

In [ ]:
collisions_by_zipcode["zip_code"] = collisions_by_zipcode["zip_code"].astype(np.string_)

In [ ]:
collisions_by_zipcode.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(x=collisions_by_zipcode[:30]["zip_code"], 
        height=collisions_by_zipcode[:30]["number_of_collisions"],
        width=0.3, color="red", edgecolor="black")
plt.xticks(rotation=90)
plt.title("Collisions by Zip Code")
plt.xlabel("Zip Codes")
plt.ylabel("Number of Collisions")
plt.grid(True)
plt.show()

## Let's scatter plot the longitudes and latitudes

In [ ]:
sql_query = " \
    SELECT longitude, latitude \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    WHERE (longitude IS NOT NULL) \
    AND (latitude IS NOT NULL) \
"

bigquery_response = client.query(sql_query)

longitude_latitude = bigquery_response.to_dataframe()

#### First let's ensure the distributions of longitudes and latitudes values make sense


In [ ]:
def clean_longitude_latitude(dataframe):
    print("Shape before removing faulty longitudes:", dataframe.shape)
    longitude_rows_to_drop = dataframe[dataframe["longitude"] < -74.27].index
    dataframe.drop(labels=longitude_rows_to_drop, inplace=True)

    longitude_rows_to_drop = dataframe[dataframe["longitude"] > -73.68].index
    dataframe.drop(labels=longitude_rows_to_drop, inplace=True)
    print("Shape after removing faulty longitudes:", dataframe.shape, "\n")
    
    print("Shape before removing faulty latitudes:", dataframe.shape)
    latitude_rows_to_drop = dataframe[dataframe["latitude"] < 40.49].index
    dataframe.drop(labels=latitude_rows_to_drop, inplace=True)

    latitude_rows_to_drop = dataframe[dataframe["latitude"] > 40.90].index
    dataframe.drop(labels=latitude_rows_to_drop, inplace=True)
    print("Shape after removing faulty latitudes:", dataframe.shape)
    
    return dataframe

In [ ]:
longitude_latitude = clean_longitude_latitude(longitude_latitude)

In [ ]:
plt.figure(figsize=(5, 3))
longitude_latitude["longitude"].hist(bins=100)
plt.title("Distribution of Longitudes")
plt.xlabel("Longitude Positions")
plt.ylabel("Number of Datapoints")
plt.show()

In [ ]:
plt.figure(figsize=(5, 3))
longitude_latitude["latitude"].hist(bins=100)
plt.title("Distribution of Latitudes")
plt.xlabel("Latitudes Positions")
plt.ylabel("Number of Datapoints")
plt.show()

In [ ]:
plt.scatter(x=longitude_latitude["longitude"], y=longitude_latitude["latitude"],
            color="green", s=10, alpha=0.002)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Heatmap of Collisions")
plt.show()

## Lethal collisions and the resulting deaths/injuries

In [ ]:
sql_query = " \
    SELECT number_of_persons_killed, COUNT(*) AS number_of_collisions \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    GROUP BY number_of_persons_killed \
    ORDER BY number_of_collisions DESC \
"

bigquery_response = client.query(sql_query)

number_of_collisions_by_deaths = bigquery_response.to_dataframe()

In [ ]:
number_of_collisions_by_deaths = number_of_collisions_by_deaths[number_of_collisions_by_deaths["number_of_persons_killed"] > 0]
number_of_collisions_by_deaths

In [ ]:
plt.figure(figsize=(6, 3))
plt.bar(x=number_of_collisions_by_deaths["number_of_persons_killed"], 
        height=number_of_collisions_by_deaths["number_of_collisions"],
        width=0.3, color="orange", edgecolor="black", linewidth=2.5)
plt.title("Deaths in Lethal Collisions")
plt.xlabel("Deaths")
plt.ylabel("Number of Collisions")
plt.grid(True)
plt.show()

## Scatter plot of where the lethal collisions most frequently occur

In [ ]:
sql_query = " \
    SELECT number_of_persons_killed, \
    number_of_pedestrians_killed, \
    number_of_cyclist_killed, \
    number_of_motorist_killed, \
    longitude, latitude \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    WHERE number_of_persons_killed > 0 \
    AND (longitude IS NOT NULL) \
    AND (latitude IS NOT NULL) \
"

bigquery_response = client.query(sql_query)

lethal_collisions = bigquery_response.to_dataframe()

In [ ]:
lethal_collisions.head()

In [ ]:
print(lethal_collisions.shape)

# drop null values
lethal_collisions.dropna(inplace=True)

# remove faulty longitude and latitude values
lethal_collisions = clean_longitude_latitude(lethal_collisions)

In [ ]:
lethal_collisions.head()

In [ ]:
lethal_collisions["number_of_persons_killed"] = lethal_collisions["number_of_persons_killed"].astype(np.float16)

In [ ]:
plt.scatter(x=lethal_collisions["longitude"], 
            y=lethal_collisions["latitude"],
            s=lethal_collisions["number_of_persons_killed"] * 10,
            color="red", edgecolor="black", alpha=0.6)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Heatmap of Lethal Collisions")
plt.grid(True)
plt.show()

## Number of pedestrians killed in lethal collisions

In [ ]:
group_by_pedestrian_deaths = lethal_collisions.groupby("number_of_pedestrians_killed") \
                                              .count().reset_index()

group_by_pedestrian_deaths = group_by_pedestrian_deaths[["number_of_pedestrians_killed", "number_of_persons_killed"]]

group_by_pedestrian_deaths.rename(columns={ "number_of_persons_killed": "number_of_lethal_collisions" }, inplace=True)

group_by_pedestrian_deaths

In [ ]:
plt.figure(figsize=(5, 3))
plt.bar(x=group_by_pedestrian_deaths["number_of_pedestrians_killed"], 
        height=group_by_pedestrian_deaths["number_of_lethal_collisions"],
        width=0.4, color="red", alpha=0.9, edgecolor="black", linewidth=2.5)
plt.title("Pedestrian Deaths in Lethal Collisions")
plt.xlabel("Number of Pedestrian Deaths")
plt.ylabel("Number of Lethal Collisions")
plt.grid(True)
plt.show()

## Number of cyclists killed in lethal collisions

In [ ]:
group_by_cyclist_deaths = lethal_collisions.groupby("number_of_cyclist_killed") \
                                           .count().reset_index()

group_by_cyclist_deaths = group_by_cyclist_deaths[["number_of_cyclist_killed", "number_of_persons_killed"]]

group_by_cyclist_deaths.rename(columns={ "number_of_persons_killed": "number_of_lethal_collisions" }, inplace=True)

group_by_cyclist_deaths

In [ ]:
plt.figure(figsize=(3, 3))
plt.bar(x=group_by_cyclist_deaths["number_of_cyclist_killed"], 
        height=group_by_cyclist_deaths["number_of_lethal_collisions"],
        width=0.4, color="red", alpha=0.9, edgecolor="black", linewidth=2.5)
plt.title("Cyclist Deaths in Lethal Collisions")
plt.xlabel("Number of Cyclist Deaths")
plt.ylabel("Number of Lethal Collisions")
plt.grid(True)
plt.show()

In [ ]:
group_by_cyclist_deaths = lethal_collisions.groupby("number_of_cyclist_killed") \
                                           .count().reset_index()

group_by_cyclist_deaths = group_by_cyclist_deaths[["number_of_cyclist_killed", "number_of_persons_killed"]]

group_by_cyclist_deaths.rename(columns={ "number_of_persons_killed": "number_of_lethal_collisions" }, inplace=True)

group_by_cyclist_deaths

## Number of motorists killed in lethal collisions

In [ ]:
plt.figure(figsize=(5, 3))
plt.bar(x=group_by_motorist_deaths["number_of_motorist_killed"], 
        height=group_by_motorist_deaths["number_of_lethal_collisions"],
        width=0.4, color="red", alpha=0.9, edgecolor="black", linewidth=2.5)
plt.title("Motorist Deaths in Lethal Collisions")
plt.xlabel("Number of Motorist Deaths")
plt.ylabel("Number of Lethal Collisions")
plt.grid(True)
plt.show()

## Graphing number of injuries caused by collisions

In [ ]:
sql_query = " \
    SELECT number_of_persons_injured, \
    number_of_pedestrians_injured, \
    number_of_cyclist_injured, \
    number_of_motorist_injured, \
    longitude, latitude \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    WHERE number_of_persons_injured > 0 \
    AND (longitude IS NOT NULL) \
    AND (latitude IS NOT NULL) \
"

bigquery_response = client.query(sql_query)

injury_collisions = bigquery_response.to_dataframe()

In [ ]:
injury_collisions.head()

In [ ]:
print(injury_collisions.shape)

# drop null values
injury_collisions.dropna(inplace=True)

# remove faulty longitude and latitude values
injury_collisions = clean_longitude_latitude(injury_collisions)

In [ ]:
injury_collisions["number_of_persons_injured"] = injury_collisions["number_of_persons_injured"].astype(np.float16)

In [ ]:
plt.scatter(x=injury_collisions["longitude"], 
            y=injury_collisions["latitude"],
            s=injury_collisions["number_of_persons_injured"],
            color="red", edgecolor="black", alpha=0.6)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Heatmap of Collisions Causing Injury")
plt.grid(True)
plt.show()

## Pedestrians injured due to collision

In [ ]:
group_by_pedestrian_injuries = injury_collisions.groupby("number_of_pedestrians_injured") \
                                                .count().reset_index()

group_by_pedestrian_injuries = group_by_pedestrian_injuries[["number_of_pedestrians_injured", "number_of_persons_injured"]]

group_by_pedestrian_injuries.rename(columns={ "number_of_persons_injured": "number_of_collisions_causing_injury" }, inplace=True)

group_by_pedestrian_injuries

In [ ]:
plt.figure(figsize=(6, 3))
plt.bar(x=group_by_pedestrian_injuries["number_of_pedestrians_injured"], 
        height=group_by_pedestrian_injuries["number_of_collisions_causing_injury"],
        width=0.4, color="blue", alpha=0.7, edgecolor="black", linewidth=2.5)
plt.title("(Log Scale) Pedestrians Injured in  Injury Causing Collisions")
plt.yscale("log")
plt.xlabel("Pedestrians Injured")
plt.ylabel("Number of Collisions (Log Scale)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.bar(x=group_by_cyclist_injuries["number_of_cyclist_injured"], 
        height=group_by_cyclist_injuries["number_of_collisions_causing_injury"],
        width=0.4, color="blue", alpha=0.7, edgecolor="black", linewidth=2.5)
plt.title("(Log Scale) Cyclists Injured in  Injury Causing Collisions")
plt.yscale("log")
plt.xlabel("Cyclists Injured")
plt.ylabel("Number of Collisions (Log Scale)")
plt.grid(True)
plt.show()

In [1]:
## Motorists injured in injury causing collisions

In [ ]:
group_by_motorist_injuries = injury_collisions.groupby("number_of_motorist_injured") \
                                             .count().reset_index()

group_by_motorist_injuries = group_by_motorist_injuries[["number_of_motorist_injured", "number_of_persons_injured"]]

group_by_motorist_injuries.rename(columns={ "number_of_persons_injured": "number_of_collisions_causing_injury" }, inplace=True)

group_by_motorist_injuries.head()

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(x=group_by_motorist_injuries["number_of_motorist_injured"], 
        height=group_by_motorist_injuries["number_of_collisions_causing_injury"],
        width=0.4, color="blue", alpha=0.7, edgecolor="black", linewidth=2.5)
plt.title("(Log Scale) Motorists Injured in  Injury Causing Collisions")
plt.yscale("log")
plt.xlabel("Motorists Injured")
plt.ylabel("Number of Collisions (Log Scale)")
plt.grid(True)
plt.show()

## First group of factors contributing to collisions

In [ ]:
sql_query = " \
    SELECT contributing_factor_vehicle_1, COUNT(*) AS number_of_collisions \
    FROM `bigquery-public-data.new_york_mv_collisions.nypd_mv_collisions` \
    GROUP BY contributing_factor_vehicle_1 \
    ORDER BY number_of_collisions DESC \
"

bigquery_response = client.query(sql_query)

first_factor_contributing_to_collisions = bigquery_response.to_dataframe()

In [ ]:
first_factor_contributing_to_collisions.head(10)

In [ ]:
plt.figure(figsize=(17, 7))
plt.bar(x=first_factor_contributing_to_collisions[:20]["contributing_factor_vehicle_1"], 
        height=first_factor_contributing_to_collisions[:20]["number_of_collisions"],
        width=0.5, color="purple", edgecolor="black", linewidth=2.5)
plt.xticks(rotation=90)
plt.yscale("log")
plt.title("(Log Scale) First Group of Factors Contributing to Collisions")
plt.xlabel("Collision Contributing Factors")
plt.ylabel("Number of Collisions (Log Scale)")
plt.grid(True)
plt.show()